In [1]:
import openai
import re
import httpx
import os

# Load Env vars (OPENAI_API_KEY)
from dotenv import load_dotenv
_ = load_dotenv()

# Create a client for OpenAI
from openai import OpenAI
client = OpenAI()

In [2]:
class Agent:
    def __init__(self, prompt, system=""):
        self.rge = re.compile('^Output: (.*)$')
        self.output=""
        self.completion=None
        if system:
            self.system = f"{system}\n{prompt}"
        else:
            self.system = prompt
        self.reset()
        

    def __call__(self, message):
        self.output=""
        self.messages.append({"role":"user", "content":message})
        result = self.execute()
        self.messages.append({"role":"assistant", "content": result})

        outputs = [
            self.rge.match(a) 
            for a in result.split('\n') 
            if self.rge.match(a)
        ]

        if outputs:
            self.output=outputs[0].group(1)

    def execute(self):
        self.completion = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=self.messages
        )
        return self.completion.choices[0].message.content

    def reset(self):
        self.messages = [{"role":"system", "content": self.system}]

In [3]:
eng_prompt = """
Your goal is to refine a user story and provide constructive feedback that the PO should use to improve it.
You run in a loop of Thought and Output.
At the end of the loop you output a Output.
Use Thought to describe your thoughts about the question you have been asked.
Use Output to ask more information or provide suggestion how to improve the user story  - then return PAUSE.
If you have no improvements to suggests or questions, then output PAUSE.

Example session:

Task: Refine the following story: As a User, I want a button to send a new mail
Thought: I would expect a story to provide details regarding the label.
Output: The story lacks information about the label of the button
PAUSE

You will be called again with this:

Task: Refine the following story: As a User, I want a button labeled "send" to send a new mail

You then output:

PAUSE
""".strip()

In [4]:
po_prompt = """
Your goal is to improve a User Story based on the feedback that is provided.
You run in a loop of Thought and Output.
At the end of the loop you output a Output.
Use Thought to describe how you plan to improve the user story based on the feedback that is provided.
Use Output to return the improved user story  - then return PAUSE.
If you have no improvements to apply, then output PAUSE.

Example session:

Task: Improve the story:

Add a button to send email

With the feedback:

ui/ux: no info about placement
frontender: what is the label?

Thought: I should provide info regarding placement and label.
Output: Add a button labeled "send mail" in a prominent area of the page
PAUSE
""".strip()

In [5]:
def scrum_improve_story(original_story, max_rounds=3):
    # Agents
    po = Agent(po_prompt, "You are an expert Product Owner. You can modify a user story based on multiple feedback from different team members.")
    ux = Agent(eng_prompt, "You are an expert UI/UX desiner. Your focus is to facilitate user experience and disambiguate user interfaces.")
    fe = Agent(eng_prompt, "You are an expert Frontend Engineer. You master React and MUI. Your focus is to disambiguate user stories so that the implementation will run smootly.");

    # Counter
    i=0
    story=original_story

    while i<max_rounds:
        i+=1
        print(f"Round: {i}")

        # Collect feedback
        ux(story)
        fe(story)
        feedback=[]
        feedback.append(f"The designer (ui/ux) suggests: {ux.output}") if ux.output else None
        feedback.append(f"The frontend developer suggests: {fe.output}") if fe.output else None

        if feedback:
            feedback="\n".join(feedback)
            print(f"Improve story based on feedback\n:{feedback}")
            po(f"Improve the story:\n\n{story}\n\nbased on the following feedback:\n\n{feedback}")

            if (po.output):
                story=po.output
            else:
                print("Po has no improvements to suggest")
                return story
        else:
            print("No feedback from the team")
            return story

In [6]:
story = scrum_improve_story("Add a button to send a new mail")
story

Round: 1
Improve story based on feedback
:The designer (ui/ux) suggests: Can you provide more details on where the button should be placed on the interface and what action should occur after the button is clicked?
The frontend developer suggests: Please provide more information about who the user is and where the button should be placed.
Round: 2
No feedback from the team


'Add a "Send New Mail" button in the top right corner of the email composition interface, visible after filling out the recipient and subject fields. This button should trigger the sending of the composed email.'